In [1]:
%load_ext autoreload
%autoreload 2
%cd -q ..

import pickle
import numpy as np
import pandas as pd
from evoaug_tf import evoaug, augment
from src.diff_expression_model import get_model, get_siamese_model, post_hoc_conjoining, get_auroc

2025-05-21 15:55:28.049218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 15:55:28.168565: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-21 15:55:28.168592: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-05-21 15:55:30.909960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

## Prepare the data

For now we start from this data pickle as I'm not aware how Helder did DE analysis and generated the labels

In [3]:
ppath = "data/dataset_solid_chrome.csv"
data = pd.read_csv(ppath)

In [4]:
data

,geneID,Category,GeneFamily,seqs,ohs,rcohs,ohsDuo,in_original_balanced,set,npshap-single,npshap-posthoc
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,[[0 0 0 1]\n [1 0 0 0]\n [0 0 1 0]\n ...\n [0 ...,[[1 0 0 0]\n [0 0 1 0]\n [0 0 1 0]\n ...\n [0 ...,[[[0 0 0 1]\n [1 0 0 0]\n [0 0 1 0]\n ...\n...,True,train,"[array([[[-2.94115947e-04],\n [ 1.11911...","[array([[[-2.47588614e-04],\n [ 1.33605..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,[[0 0 1 0]\n [0 0 0 1]\n [0 1 0 0]\n ...\n [0 ...,[[0 0 1 0]\n [0 0 1 0]\n [1 0 0 0]\n ...\n [0 ...,[[[0 0 1 0]\n [0 0 0 1]\n [0 1 0 0]\n ...\n...,True,valid,"[array([[[ 4.96114546e-04],\n [ 6.92353...","[array([[[ 5.19316818e-04],\n [ 3.21869..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,[[1 0 0 0]\n [0 0 1 0]\n [0 0 0 1]\n ...\n [0 ...,[[0 0 0 1]\n [0 0 1 0]\n [1 0 0 0]\n ...\n [1 ...,[[[1 0 0 0]\n [0 0 1 0]\n [0 0 0 1]\n ...\n...,True,valid,"[array([[[-3.02427739e-04],\n [-1.08283...","[array([[[-3.14956094e-04],\n [-8.34961..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,[[0 1 0 0]\n [1 0 0 0]\n [0 1 0 0]\n ...\n [0 ...,[[1 0 0 0]\n [0 1 0 0]\n [0 0 1 0]\n ...\n [0 ...,[[[0 1 0 0]\n [1 0 0 0]\n [0 1 0 0]\n ...\n...,True,train,"[array([[[ 2.54294393e-03],\n [ 6.39127...","[array([[[ 2.08252599e-03],\n [ 1.86260..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,[[0 0 0 1]\n [0 0 0 1]\n [0 0 1 0]\n ...\n [1 ...,[[0 0 0 1]\n [1 0 0 0]\n [0 0 0 1]\n ...\n [0 ...,[[[0 0 0 1]\n [0 0 0 1]\n [0 0 1 0]\n ...\n...,True,valid,"[array([[[-5.23597009e-05],\n [ 4.53725...","[array([[[ 5.18305111e-04],\n [ 5.07344..."
...,...,...,...,...,...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,[[1 0 0 0]\n [1 0 0 0]\n [0 0 1 0]\n ...\n [1 ...,[[0 1 0 0]\n [0 0 1 0]\n [0 0 0 1]\n ...\n [0 ...,[[[1 0 0 0]\n [1 0 0 0]\n [0 0 1 0]\n ...\n...,True,test,"[array([[[-1.85048248e-05],\n [ 5.31432...","[array([[[ 3.42493469e-04],\n [ 3.91582..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,[[0 0 0 1]\n [0 0 0 1]\n [0 0 0 1]\n ...\n [0 ...,[[0 0 0 1]\n [0 0 1 0]\n [1 0 0 0]\n ...\n [1 ...,[[[0 0 0 1]\n [0 0 0 1]\n [0 0 0 1]\n ...\n...,True,train,"[array([[[ 9.53665585e-05],\n [ 3.28118...","[array([[[-5.87131144e-05],\n [ 6.68894..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,[[0 0 0 1]\n [1 0 0 0]\n [0 0 0 1]\n ...\n [0 ...,[[1 0 0 0]\n [0 0 1 0]\n [0 0 1 0]\n ...\n [1 ...,[[[0 0 0 1]\n [1 0 0 0]\n [0 0 0 1]\n ...\n...,True,test,"[array([[[ 6.18251797e-04],\n [ 6.60565...","[array([[[ 4.48545878e-04],\n [ 6.36329..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,[[0 0 0 1]\n [0 0 1 0]\n [0 0 1 0]\n ...\n [0 ...,[[0 0 0 1]\n [0 0 0 1]\n [0 0 1 0]\n ...\n [0 ...,[[[0 0 0 1]\n [0 0 1 0]\n [0 0 1 0]\n ...\n...,True,train,"[array([[[-2.15267210e-04],\n [ 3.76152...","[array([[[-1.03627557e-04],\n [ 1.56308..."


In [5]:
data["set"].value_counts()

train    1900
valid     257
test      241
Name: set, dtype: int64

To show how to subdivide the dataset into train-test split we only take the following columns

In [6]:
dataset = data.reset_index()
dataset = dataset[["geneID", "Category", "GeneFamily", "seqs"]]
dataset.rename(columns={"geneID":"GeneID", "Category":"Label", "seqs": "Sequence"}, inplace=True)

In [7]:
dataset

,GeneID,Label,GeneFamily,Sequence
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...
...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...


## Phylo augmentation 

Define the fasta files for each species of which we want to use orthologs.

In [8]:
phylo_augmentation_base_path = "data/phylo_aug/fasta/"
species_fasta_files = {
			"aly":"aly.tss.fasta",
			"chi":"chi.tss.fasta",
			"cru":"cru.tss.fasta"
}

species_to_include = ["aly","chi","cru"]

Parsing the fasta files. In this case the label of the original sequence is incorporated in the ids in the fasta file
e.g:
```
>Aa31LG6G5540_1
TTGTATAAATATATTAAATT
```

In [9]:
from Bio import SeqIO

sequence_list = []
gene_id_list = []
label_list = []
species_list = []
for species in species_to_include:
	species_fasta = species_fasta_files[species]
	fasta_path = f"{phylo_augmentation_base_path}{species_fasta}"
	fasta_sequences = SeqIO.parse(open(fasta_path),'fasta')
	print(f"File: {fasta_path} done!")
	for fasta in fasta_sequences:
		name, sequence = fasta.id, str(fasta.seq)
		
		# The label is stored inside the id name (e.g. >ATERI-4G42060_1), split these
		gene_id = name.split("_")[0]
		label = name.split("_")[1]
		species = species_fasta.split(".")[0]

		gene_id_list.append(gene_id)
		sequence_list.append(sequence)
		label_list.append(label)
		species_list.append(species)

File: data/phylo_aug/fasta/aly.tss.fasta done!
File: data/phylo_aug/fasta/chi.tss.fasta done!
File: data/phylo_aug/fasta/cru.tss.fasta done!


In [10]:
phylo_aug_df = pd.DataFrame({"GeneID": gene_id_list, "Label": label_list, "Sequence": sequence_list, "Species": species_list})

In [11]:
phylo_aug_df

,GeneID,Label,Sequence,Species
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly
...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru


### Get gene family of the original ortholog

Because during the batch time augmentation a original sample will be swapped by one of its orthologs, we need to know from which original sample each ortholog originates from.

In [12]:
base_dir_ortholog = "data/phylo_aug/orthologs/"
ortholog_files = {
			"ath-aly":"ath_aly.txt",
			"ath-chi":"ath_chi.txt",
			"ath-cru":"ath_cru.txt"
			}

Create a lookup table to include the orthologs in the dataframe

In [13]:
lookup_table = {}
for orthologs in ortholog_files.values():
	with open(f"{base_dir_ortholog}{orthologs}", 'r') as file:
		for line in file:
			# Split each line by tab
			columns = line.strip().split('\t')
			if len(columns) >= 2:  # Ensure there are at least two columns
				key = columns[1]
				value = columns[0]
				lookup_table[key] = value

In [14]:
phylo_aug_df["Ortholog"] = phylo_aug_df["GeneID"].map(lookup_table)

In [15]:
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260
...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530


In [16]:
phylo_aug_df = pd.merge(phylo_aug_df, dataset[["GeneID", "GeneFamily"]], left_on="Ortholog", right_on="GeneID", how="left")

In [17]:
phylo_aug_df.drop("GeneID_y", axis=1, inplace=True)
phylo_aug_df.rename(columns={"GeneID_x": "GeneID"}, inplace=True)
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130
...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279


### One-hot-encode & reverse-complement both dataset as augmented dataset

In [18]:
from src.prepare_dataset import one_hot_encode_series, reverse_complement_series, reverse_complement_sequence
dataset["One_hot_encoded"] = one_hot_encode_series(dataset["Sequence"])
phylo_aug_df["One_hot_encoded"] = one_hot_encode_series(phylo_aug_df["Sequence"])

In [19]:
dataset["RC_one_hot_encoded"] = reverse_complement_series(dataset["One_hot_encoded"])
phylo_aug_df["RC_one_hot_encoded"] = reverse_complement_series(phylo_aug_df["One_hot_encoded"])

In [20]:
dataset

,GeneID,Label,GeneFamily,Sequence,One_hot_encoded,RC_one_hot_encoded
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,..."
...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,..."


In [21]:
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130,"[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1,..."
...,...,...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357,"[[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


In [22]:
phylo_aug_df = phylo_aug_df[phylo_aug_df["GeneFamily"].notna()]
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
4,AL1G44210,1,CGGCAATGTAATTAATACCAAAACCAAATGGGCGGCACTACCCGCT...,aly,AT1G30260,HOM04D006130,"[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1,..."
...,...,...,...,...,...,...,...,...
2789,Carub.0004s0553,0,AATCCAACGGCATGAGTACATGGGTTTCGAGAACCTTTGCAAGTAC...,cru,AT2G25740,HOM04D004133,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2791,Carub.0005s2520,0,GCAGCTGAACTATTGGGTGTTCGTCCTTGTGTCTTTAAGCCTCTTC...,cru,AT3G58460,HOM04D006357,"[[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


### Train-test split

In [23]:
from src.prepare_dataset import grouped_shuffle_split
train_df, validation_test_df = grouped_shuffle_split(dataset, dataset["GeneFamily"], 0.2)

In [24]:
validation_df, test_df  = grouped_shuffle_split(validation_test_df, validation_test_df["GeneFamily"], 0.5)

In [25]:
print(f"Length of training set: {train_df.shape[0]}")
print(f"Length of validation set: {validation_df.shape[0]}")
print(f"Length of test set: {test_df.shape[0]}")

Length of training set: 1900
Length of validation set: 257
Length of test set: 241


## Append train and augmentation dataframe

Filter out the orthologs from samples in test or validation

In [26]:
filtered_phylo_aug_df = phylo_aug_df[~phylo_aug_df["Ortholog"].isin(validation_test_df["GeneID"])]

In [27]:
filtered_phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily,One_hot_encoded,RC_one_hot_encoded
0,AL1G34890,0,AACTGACACCTAAACTCCACTGCAGAGTGGCAGAAGCATTTGTAAG...,aly,AT1G21720,HOM04D003099,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
1,AL6G48600,1,CCTACGCCACATTAACCTGATCATACCTAAACACGGTGAATCATCA...,aly,AT4G05010,HOM04D003060,"[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1,..."
2,AL6G13960,1,TTAGTCGATGCTTTATTGAAAAATATATAGTTTGTAAAACAGTCAT...,aly,AT5G04340,HOM04D000135,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,...","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
3,AL4G45030,1,ATCGATGTAGATATTATCTTATTCTTATACATTTAATACAGATTCA...,aly,AT2G45820,HOM04D001302,"[[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
5,AL6G25970,0,GATTGAGAGTGGGTGTCATAGTTTCTTTTAATTTTTTTGCTTATCT...,aly,AT5G15150,HOM04D000209,"[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
...,...,...,...,...,...,...,...,...
2787,Carub.0005s1225,1,GAAAGAATGGAGAGAGAAGAGAGACGAGTTCAAGAAGAAAGTGAGC...,cru,AT3G46450,HOM04D003906,"[[0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...","[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."
2788,Carub.0005s2556,1,TTGTGTAGTGCCTTGAAAACGTCCTGATCCCTCCAAAGATGTTACA...,cru,AT3G58780,HOM04D000037,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2790,Carub.0006s3101,1,TTCTTTCTCTTTCTCACTATAATAAATATATAATGAAAAAAAAATT...,cru,AT4G11910,HOM04D001924,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
2792,Carub.0005s2528,0,GCGCAAGAAGCGGAGAACAGGGACGGTGAGACCGAGGAGTTTTTGC...,cru,AT3G58530,HOM04D007279,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0,...","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."


## Batch time augmentation 

In [28]:
import random

def get_batch(data, phylo_aug_data, indices, perform_phylo_aug=True, phylo_aug_rate=1.0):
	"""
	Creates a batch of the input and one-hot encodes the sequences
	"""
	seqs = []
	labels = []
	gene_ids = []
	for ii in indices:
		gene_id = data.iloc[ii].GeneID
		sample = data.iloc[ii]
		if random.choices(population=[0, 1], weights=[1 - phylo_aug_rate, phylo_aug_rate])[0] == 1:
			orthologs = phylo_aug_data[phylo_aug_data.Ortholog == gene_id]
			if not orthologs.empty:
				sample = phylo_aug_data[phylo_aug_data.Ortholog == gene_id].sample().iloc[0]
			# Randomly pick the forward or reverse complement strand
		if random.randint(0, 1) == 0:
			gene_ids.append(sample.GeneID)
			seqs.append(sample.RC_one_hot_encoded)
		else:
			gene_ids.append(sample.GeneID)
			seqs.append(sample.One_hot_encoded)
			
		labels.append(sample.Label)

	X_batch = np.array(seqs)
	X_batch = X_batch.astype('float32')

	Y_batch = np.array(labels)
	Y_batch = Y_batch.astype('int64')

	return X_batch, Y_batch

In [29]:
def data_gen(data, phylo_aug_data, batch_size, perform_phylo_aug=True, phylo_aug_rate=1.0):
	"""
	Generator function for loading input data in batches
	"""
	num_samples = data.shape[0]
	# indices = list(range(num_samples))
	indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)
	# indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)

	ii = 0
	while True:
		yield get_batch(data, phylo_aug_data, indices[ii:ii + batch_size], perform_phylo_aug, phylo_aug_rate)
		ii += batch_size
		if ii >= num_samples:
			ii = 0
			indices = np.random.choice(list(range(num_samples)), num_samples, replace=False)
			# indices = list(range(num_samples))


## Prepare validation input

In [30]:
def get_input_and_labels(df):
	ohe_np = np.stack(df["One_hot_encoded"])
	rc_np = np.stack(df["RC_one_hot_encoded"])

	x = np.append(ohe_np, rc_np, axis=0)
	x = x.astype('float32')
	y = np.append(df["Label"], df["Label"])
	y = y.astype('int64')
	return x, y

In [31]:
x_validation, y_validation = get_input_and_labels(validation_df)

As we do batch time augmentation we define the data generator here

In [32]:
BATCH_SIZE = 200
train_data_gen = data_gen(train_df,filtered_phylo_aug_df, BATCH_SIZE, True, 0.7)

## Prepare model

Get the shape of the input data

In [33]:
input_shape = train_df["One_hot_encoded"].iloc[0].shape

Initialize the model

In [34]:
model = get_model(input_shape=input_shape, perform_evoaug=False,learning_rate=0.001)

2025-05-21 15:56:19.690170: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-05-21 15:56:19.690211: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-21 15:56:19.690242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-vibcompute-1.vib.local): /proc/driver/nvidia/version does not exist
2025-05-21 15:56:19.690564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Train the model

In [35]:
# early stopping callback
import tensorflow as tf

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=40,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# reduce learning rate callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
												factor=0.1,
												patience=5,
												min_lr=1e-7,
												mode='min',
												verbose=1)
callbacks = [early_stopping_callback, reduce_lr]

In [36]:
import math
history = model.fit(train_data_gen,
					steps_per_epoch=math.ceil(
						train_df.shape[0] / BATCH_SIZE),
					epochs=100,
					validation_data=(x_validation, y_validation),
					callbacks=callbacks
					)

Epoch 1/100
10/10 [==============================] - 5s 305ms/step - loss: 0.6955 - acc: 0.4947 - auROC: 0.4725 - auPRC: 0.4758 - true_positives: 154.0000 - precision: 0.4695 - recall: 0.1638 - val_loss: 0.6933 - val_acc: 0.5039 - val_auROC: 0.5158 - val_auPRC: 0.5307 - val_true_positives: 249.0000 - val_precision: 0.5030 - val_recall: 0.9651 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 2s 246ms/step - loss: 0.6942 - acc: 0.4911 - auROC: 0.4936 - auPRC: 0.4847 - true_positives: 560.0000 - precision: 0.4882 - recall: 0.5957 - val_loss: 0.6930 - val_acc: 0.5233 - val_auROC: 0.5550 - val_auPRC: 0.5509 - val_true_positives: 32.0000 - val_precision: 0.6275 - val_recall: 0.1240 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 2s 239ms/step - loss: 0.6943 - acc: 0.5058 - auROC: 0.4866 - auPRC: 0.4937 - true_positives: 212.0000 - precision: 0.5012 - recall: 0.2255 - val_loss: 0.6934 - val_acc: 0.4981 - val_auROC: 0.5519 - val_auPRC: 0.5299 - val_true_posit

In [37]:
scores = model.evaluate(x_validation, y_validation, verbose=0)
print(f'Score: Loss of {scores[0]}; accuracy of {scores[1]}; auroc of {scores[2]}; auprc of {scores[3]}; TP of {scores[4]}')

Score: Loss of 0.6434972286224365; accuracy of 0.6284046769142151; auroc of 0.6885219812393188; auprc of 0.6634527444839478; TP of 164.0


### Finetune phylo-aug

In [38]:
x_train_pure, y_train_pure = get_input_and_labels(train_df)


In [39]:
from tensorflow import keras
from src.diff_expression_model import compile_model

# set up callbacks
model = compile_model(model, learning_rate=0.0001)

# train model
model.fit(x_train_pure, y_train_pure,
				epochs=10,
				batch_size=100,
				shuffle=True,
				validation_data=(x_validation, y_validation),
				callbacks=[early_stopping_callback])

Epoch 1/10
38/38 [==============================] - 4s 63ms/step - loss: 0.5985 - acc: 0.6824 - auROC: 0.7413 - auPRC: 0.7063 - true_positives_1: 1313.0000 - precision_1: 0.6723 - recall_1: 0.6984 - val_loss: 0.6441 - val_acc: 0.6401 - val_auROC: 0.6875 - val_auPRC: 0.6582 - val_true_positives_1: 185.0000 - val_precision_1: 0.6229 - val_recall_1: 0.7171
Epoch 2/10
38/38 [==============================] - 2s 50ms/step - loss: 0.5886 - acc: 0.6882 - auROC: 0.7541 - auPRC: 0.7171 - true_positives_1: 1344.0000 - precision_1: 0.6744 - recall_1: 0.7149 - val_loss: 0.6474 - val_acc: 0.6459 - val_auROC: 0.6875 - val_auPRC: 0.6544 - val_true_positives_1: 196.0000 - val_precision_1: 0.6203 - val_recall_1: 0.7597
Epoch 3/10
38/38 [==============================] - 2s 49ms/step - loss: 0.5826 - acc: 0.6908 - auROC: 0.7604 - auPRC: 0.7246 - true_positives_1: 1355.0000 - precision_1: 0.6758 - recall_1: 0.7207 - val_loss: 0.6449 - val_acc: 0.6381 - val_auROC: 0.6902 - val_auPRC: 0.6583 - val_true_pos

In [42]:
scores = model.evaluate(x_validation, y_validation, verbose=0)
print(f'Score: Loss of {scores[0]}; accuracy of {scores[1]}; auroc of {scores[2]}; auprc of {scores[3]}; TP of {scores[4]}')

Score: Loss of 0.6562544703483582; accuracy of 0.6420233249664307; auroc of 0.6844037771224976; auprc of 0.6436824202537537; TP of 199.0


## Test set

We perform post-hoc conjoing

In [44]:
siamese_model = get_siamese_model(model)

Prepare the test set data

In [45]:
x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

In [46]:
y_test = test_df["Label"].to_numpy()

In [49]:
predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

8/8 [==============================] - 0s 8ms/step


In [50]:
get_auroc(y_test, predictions)

0.7494802494802495

In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_categories))

              precision    recall  f1-score   support

           0       0.70      0.56      0.62       111
           1       0.68      0.80      0.73       130

    accuracy                           0.69       241
   macro avg       0.69      0.68      0.68       241
weighted avg       0.69      0.69      0.68       241

